In [1]:
import torch
import numpy as np
from Networks.mesh2mesh import *
from Networks.rgbd2mesh import RGBD2Mesh
import trimesh
from Networks.obj2pointcloud import *
from Networks.pointclouddataset import *

In [2]:
####1 get data for stage one autoencoder training

train_dataset = pcd_stage1(split='train')
train_dataloader = torch.utils.data.DataLoader(
        train_dataset,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=1,   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True  # This is an implementation detail to speed up data uploading to the GPU
    )
val_dataset = pcd_stage1(split='val')
val_dataloader = torch.utils.data.DataLoader(
        val_dataset,     # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=1,   # The size of batches is defined here
        shuffle=False,   # During validation, shuffling is not necessary anymore
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True  # This is an implementation detail to speed up data uploading to the GPU
    )    

In [2]:
config = {
    'experiment_name': 'pcd_stage1_overfit',
    'device': 'cuda:0',                   # change this to cpu if you do not have a GPU
    'is_overfit': True,                   # True since we're doing overfitting
    'batch_size': 1,
    'resume_ckpt_en': None,
    'resume_ckpt_de': None,
    'learning_rate': 0.001,
    'max_epochs': 300,
    'print_every_n': 10,
    'validate_every_n': 10,
}

In [3]:
from Networks import train_mesh2mesh


train_mesh2mesh.main(config)


Using device: cuda:0
[000/00009] train_loss: 288.740
[000/00009] val_loss: 212.146
[001/00009] train_loss: 337.432
[001/00009] val_loss: 204.462
[002/00009] train_loss: 227.578
[002/00009] val_loss: 217.426
[003/00009] train_loss: 219.915
[003/00009] val_loss: 188.284
[004/00009] train_loss: 187.207
[004/00009] val_loss: 156.776
[005/00009] train_loss: 190.598
[005/00009] val_loss: 157.789
[006/00009] train_loss: 168.000
[006/00009] val_loss: 140.319
[007/00009] train_loss: 149.344
[007/00009] val_loss: 148.574
[008/00009] train_loss: 175.154
[008/00009] val_loss: 131.206
[009/00009] train_loss: 139.257
[009/00009] val_loss: 195.801
[010/00009] train_loss: 206.207
[010/00009] val_loss: 194.420
[011/00009] train_loss: 180.496
[011/00009] val_loss: 147.374
[012/00009] train_loss: 126.813
[012/00009] val_loss: 110.614
[013/00009] train_loss: 108.884
[013/00009] val_loss: 70.886
[014/00009] train_loss: 61.532
[014/00009] val_loss: 36.940
[015/00009] train_loss: 28.789
[015/00009] val_loss:

KeyboardInterrupt: 

In [5]:
encoder = PointNetEncoder(numpoints=40000)
encoder.load_state_dict(torch.load(f'./models/runs/{config["experiment_name"]}/encoder_best.pth'))
encoder.eval()
decoder = PointNetDecoder(numpoints=40000)
decoder.load_state_dict(torch.load(f'./models/runs/{config["experiment_name"]}/decoder_best.pth'))
decoder.eval()
x = 1
for batch in val_dataloader:
    input = batch.numpy()
    batch = batch.permute(0,2,1)
    recon = decoder(encoder(batch))
    out = recon.permute(0,2,1).detach().numpy()

    print(input.shape)
    print(out.shape)

    
    export_pointcloud_to_obj(f'./input_{x}.obj', input[0])
    export_pointcloud_to_obj(f'./output_{x}.obj', out[0])


    x +=1
    if x==10: break

    



(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)


In [5]:
encoder = PointNetEncoder(numpoints=40000)
encoder.load_state_dict(torch.load(f'./models/runs/{config["experiment_name"]}/encoder_best.pth'))
encoder.eval()
decoder = PointNetDecoder(numpoints=40000)
decoder.load_state_dict(torch.load(f'./models/runs/{config["experiment_name"]}/decoder_best.pth'))
decoder.eval()
val_dataset = pcd_stage1(split='overfit')
val_dataloader = torch.utils.data.DataLoader(
        val_dataset,     # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=1,   # The size of batches is defined here
        shuffle=False,   # During validation, shuffling is not necessary anymore
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True  # This is an implementation detail to speed up data uploading to the GPU
    )   
x=1    
for batch in val_dataloader:
    input = batch.numpy()
    batch = batch.permute(0,2,1)
    recon = decoder(encoder(batch))
    out = recon.permute(0,2,1).detach().numpy()

    print(input.shape)
    print(out.shape)

    
    export_pointcloud_to_obj(f'./input_{x}.obj', input[0])
    export_pointcloud_to_obj(f'./output_{x}.obj', out[0])


    x +=1
    if x==10: break    

(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
(1, 40000, 3)
